In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import glob

In [ ]:
data_path = '/content/drive/My Drive/Mineria de datos/datasets/ready/'

training_sets_paths = sorted(glob.glob(data_path + "training*.pkl"))
test_sets_paths = sorted(glob.glob(data_path + 'test*.pkl'))

In [ ]:
dataset_index = -1

In [ ]:
TRAINING = pd.read_pickle(training_sets_paths[dataset_index])
TEST = pd.read_pickle(test_sets_paths[dataset_index])

In [ ]:
TRAINING

,session_id,context_switch1,no_pause_before_play1,short_pause_before_play1,long_pause_before_play1,hist_user_behavior_n_seekfwd1,hist_user_behavior_n_seekback1,hist_user_behavior_is_shuffle1,hour_of_day1,premium1,context_type1,hist_user_behavior_reason_start1,hist_user_behavior_reason_end1,duration1,us_popularity_estimate1,mode1,pca11,pca21,pca31,pca41,pca51,pca61,pca71,pca81,pca91,pca101,pca111,pca121,pca131,pca141,pca151,context_switch2,no_pause_before_play2,short_pause_before_play2,long_pause_before_play2,hist_user_behavior_n_seekfwd2,hist_user_behavior_n_seekback2,hist_user_behavior_is_shuffle2,hour_of_day2,premium2,context_type2,hist_user_behavior_reason_start2,hist_user_behavior_reason_end2,duration2,us_popularity_estimate2,mode2,pca12,pca22,pca32,pca42,pca52,pca62,pca72,pca82,pca92,pca102,pca112,pca122,pca132,pca142,pca152,session_position,session_length,skip
0,12_655db710-19f6-41a1-be03-4d84d0856493,False,False,False,False,0.0,0.000000,False,0.434783,True,user_collection,trackdone,backbtn,0.151525,0.998968,minor,-2.518307,0.353835,0.372552,1.050369,-1.409170,0.247857,-0.519959,-1.985823,-1.394479,0.120473,-0.278987,0.613380,-0.006562,0.109401,-0.434673,False,False,False,False,0.0,0.000000,False,0.434783,True,user_collection,trackdone,backbtn,0.151525,0.998968,minor,-2.518307,0.353835,0.372552,1.050369,-1.409170,0.247857,-0.519959,-1.985823,-1.394479,0.120473,-0.278987,0.613380,-0.006562,0.109401,-0.434673,1,20,False
1,12_655db710-19f6-41a1-be03-4d84d0856493,False,False,False,False,0.0,0.000000,False,0.434783,True,user_collection,trackdone,backbtn,0.151525,0.998968,minor,-2.518307,0.353835,0.372552,1.050369,-1.409170,0.247857,-0.519959,-1.985823,-1.394479,0.120473,-0.278987,0.613380,-0.006562,0.109401,-0.434673,False,True,False,False,0.0,0.000000,False,0.434783,True,user_collection,backbtn,trackdone,0.083050,0.999982,minor,-4.259579,1.942952,1.446588,0.462302,-0.969078,-0.585451,1.183217,-0.588779,-0.083432,1.081676,-0.835342,-0.827247,-0.425448,0.769361,-0.694590,2,20,False
2,12_655db710-19f6-41a1-be03-4d84d0856493,False,True,False,False,0.0,0.000000,False,0.434783,True,user_collection,backbtn,trackdone,0.083050,0.999982,minor,-4.259579,1.942952,1.446588,0.462302,-0.969078,-0.585451,1.183217,-0.588779,-0.083432,1.081676,-0.835342,-0.827247,-0.425448,0.769361,-0.694590,False,True,False,False,0.0,0.000000,False,0.434783,True,user_collection,trackdone,trackdone,0.151525,0.998968,minor,-2.518307,0.353835,0.372552,1.050369,-1.409170,0.247857,-0.519959,-1.985823,-1.394479,0.120473,-0.278987,0.613380,-0.006562,0.109401,-0.434673,3,20,False
3,12_655db710-19f6-41a1-be03-4d84d0856493,False,True,False,False,0.0,0.000000,False,0.434783,True,user_collection,trackdone,trackdone,0.151525,0.998968,minor,-2.518307,0.353835,0.372552,1.050369,-1.409170,0.247857,-0.519959,-1.985823,-1.394479,0.120473,-0.278987,0.613380,-0.006562,0.109401,-0.434673,False,True,False,False,0.0,0.000000,False,0.434783,True,user_collection,trackdone,trackdone,0.090558,0.999977,major,-4.676861,1.028377,0.882011,1.208987,-0.354563,-0.267258,1.938937,-1.386924,-0.171904,0.961702,-0.730743,-0.205703,-0.518115,0.353632,-0.447203,4,20,False
4,12_655db710-19f6-41a1-be03-4d84d0856493,False,True,False,False,0.0,0.000000,False,0.434783,True,user_collection,trackdone,trackdone,0.090558,0.999977,major,-4.676861,1.028377,0.882011,1.208987,-0.354563,-0.267258,1.938937,-1.386924,-0.171904,0.961702,-0.730743,-0.205703,-0.518115,0.353632,-0.447203,False,False,True,True,0.0,0.000000,False,0.434783,True,user_collection,trackdone,fwdbtn,0.085617,0.998840,major,-2.984156,1.148004,1.204549,0.857785,-1.196619,0.128140,-0.864100,0.884927,-0.198011,0.971740,-0.478327,-0.245269,-0.329754,0.189268,-0.525744,5,20,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5272123,12_53

### One Hot Encoding

In [ ]:
columns_fe = []

for i in range(1, 3):
    columns_fe.append('hist_user_behavior_reason_start'+str(i))
    columns_fe.append('hist_user_behavior_reason_end'+str(i))
    columns_fe.append('mode'+str(i))
    columns_fe.append('context_type'+str(i))

In [ ]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

ENCODED_training = []
ENCODED_test = []
for column in columns_fe:
    le = LabelEncoder()
    ohe = OneHotEncoder()

    le.fit(TRAINING[column])
    le.fit(TEST[column])

    TEMP_train = le.transform(TRAINING[column])
    TEMP_test  = le.transform(TEST[column])

    ohe.fit(TEMP_train.reshape(-1,1))
    ohe.fit(TEMP_test.reshape(-1,1))

    X_train = ohe.transform(TEMP_train.reshape(-1,1)).toarray()
    X_test  = ohe.transform(TEMP_test.reshape(-1,1)).toarray()


    ENCODED_training.append(
        pd.DataFrame(X_train, columns=[column+str(i) for i in range(X_train.shape[1])])
    )

    ENCODED_test.append(
        pd.DataFrame(X_test, columns=[column+str(i) for i in range(X_test.shape[1])])
    )

In [ ]:
ENCODED_training

[         hist_user_behavior_reason_start10  ...  hist_user_behavior_reason_start19
 0                                      0.0  ...                                0.0
 1                                      0.0  ...                                0.0
 2                                      0.0  ...                                0.0
 3                                      0.0  ...                                0.0
 4                                      0.0  ...                                0.0
 ...                                    ...  ...                                ...
 5272123                                0.0  ...                                0.0
 5272124                                0.0  ...                                0.0
 5272125                                0.0  ...                                0.0
 5272126                                0.0  ...                                0.0
 5272127                                0.0  ...                            

In [ ]:
TRAINING = pd.concat([TRAINING] + ENCODED_training, axis=1)
TEST = pd.concat([TEST] + ENCODED_test, axis=1)

In [ ]:
TRAINING.drop(columns_fe, axis=1, inplace=True)
TEST.drop(columns_fe, axis=1, inplace=True)

In [ ]:
TRAINING

## Training

### Support Vector Machines

Try:
- **Kernel:** linear | poly | rbf | sigmoid | precomputed
- **shrinking:** True | False

### Naive Bayes

In [ ]:
from sklearn.naive_bayes import ClassifierMixin
from sklearn import preprocessing
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn import metrics




In [ ]:
features_column_names = list(TRAINING.columns)
features_column_names.remove('skip')
features_column_names.remove('session_length')
features_column_names.remove('session_position')
features_column_names.remove('session_id')

In [ ]:
#Prueba con DataSetDummy
weather=['Sunny','Sunny','Overcast','Rainy','Rainy','Rainy','Overcast','Sunny','Sunny',
         'Rainy','Sunny','Overcast','Overcast','Rainy']
temp=['Hot','Hot','Hot','Mild','Cool','Cool','Cool','Mild','Cool','Mild','Mild','Mild','Hot','Mild']

play=['No','No','Yes','Yes','Yes','No','Yes','No','Yes','Yes','Yes','Yes','Yes','No']

le = preprocessing.LabelEncoder()
weather_encoded=le.fit_transform(weather)
temp_encoded=le.fit_transform(temp)
label=le.fit_transform(play)

"""print(weather_encoded)
print(temp_encoded)
print(label)"""

features=zip(weather_encoded,temp_encoded)
features2= np.array(list(features))
print (list(features))
print(features2)

#Sigo con la prueba

model = GaussianNB()
model.fit(list(features2),label)
predicted= model.predict([[0,2]])
print ("Predicted Value:"+ str(predicted))


[]
[[2 1]
 [2 1]
 [0 1]
 [1 2]
 [1 0]
 [1 0]
 [0 0]
 [2 2]
 [2 0]
 [1 2]
 [2 2]
 [0 2]
 [0 1]
 [1 2]]
Predicted Value:[1]


In [ ]:
#Prueba con Multilabeles

from sklearn import datasets

wine = datasets.load_wine()
print(wine.feature_names)
print(wine.target_names)

X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.3,random_state=109) 
# 70% training and 30% test

gnb = GaussianNB()
gnb.fit(X_train, y_train)
y_pred = gnb.predict(X_test)
print(y_pred)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))


['alcohol', 'malic_acid', 'ash', 'alcalinity_of_ash', 'magnesium', 'total_phenols', 'flavanoids', 'nonflavanoid_phenols', 'proanthocyanins', 'color_intensity', 'hue', 'od280/od315_of_diluted_wines', 'proline']
['class_0' 'class_1' 'class_2']
[0 0 1 2 0 1 0 0 1 0 2 2 2 2 0 1 1 0 0 1 2 1 0 2 0 0 1 2 0 1 2 1 1 0 1 1 0
 2 2 0 2 1 0 0 0 2 2 0 1 1 2 0 0 2]
Accuracy: 0.9074074074074074


### Decision Tree Classifier

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

### Ensemble: Bagging Decision Trees

In [ ]:
from sklearn.ensemble import BaggingClassifier

### Ensemble: Gradient Boosting trees.

In [ ]:
features_column_names = list(TRAINING.columns)
features_column_names.remove('skip')
features_column_names.remove('session_length')
features_column_names.remove('session_position')
features_column_names.remove('session_id')

In [ ]:
import xgboost as xgb

xg_model = xgb.XGBClassifier(
    eta=0.1,
    max_depth=11,
    n_estimators=100,
    tree_method='gpu_hist'
)

In [ ]:
train_indexes = list(TRAINING.index.values)
train_indexes_len = len(train_indexes)
split_size = int(train_indexes_len/8)

In [ ]:
train_splits = []
for i in range(7):
    train_splits.append(train_indexes[:split_size])
    del train_indexes[:split_size]
train_splits.append(train_indexes)

In [ ]:
xg_model.fit(TRAINING.loc[train_splits[0], features_column_names], TRAINING.loc[train_splits[0], 'skip'])

In [ ]:
result = xg_model.predict(TEST[features_column_names])

### GroundTruth

In [ ]:
def get_ground_truth(input_df):
    ground_truths = []     
    df = input_df.reset_index()
    current_index = 0
    # Here we process each session.
    while current_index < len(df):
        partial_length = df['session_length'].iloc[current_index]-df['session_position'].iloc[current_index]+1
        session_skips = list(df.loc[current_index:current_index+partial_length-1, 'skip'])
        ground_truths.append(session_skips)
        current_index += partial_length 
    return ground_truths


def evaluate(submission,groundtruth):
    ap_sum = 0.0
    first_pred_acc_sum = 0.0
    counter = 0
    for sub, tru in zip(submission, groundtruth):
        if len(sub) != len(tru):
            raise Exception('Line {} should contain {} predictions, but instead contains '
                            '{}'.format(counter+1,len(tru),len(sub)))
        ap_sum += ave_pre(sub,tru,counter)
        first_pred_acc_sum += sub[0] == tru[0]
        counter+=1
    ap = ap_sum/counter
    first_pred_acc = first_pred_acc_sum/counter
    return ap,first_pred_acc

def ave_pre(submission,groundtruth,counter):
    s = 0.0
    t = 0.0
    c = 1.0
    for x, y in zip(submission, groundtruth):
        if x != 0 and x != 1:
            raise Exception('Invalid prediction in line {}, should be 0 or 1'.format(counter))
        if x==y:
            s += 1.0
            t += s / c
        c += 1
    return t/len(groundtruth)


In [ ]:
gt = get_ground_truth(TEST[['skip', 'session_length', 'session_position']])

In [ ]:
pred = get_ground_truth(pd.concat(
    [
        TEST[['session_length', 'session_position']],
        pd.DataFrame(result, columns=['skip'])
     ], 
     axis=1)
)

In [ ]:
a, b = evaluate(pred, gt)

In [ ]:
print(a)
print(b)

0.7933318204365721
0.8389650193773905


In [ ]:
cm = pd.crosstab(TEST['skip'].values, result)

In [ ]:
acc = (cm.iloc[0,0] + cm.iloc[1,1])/ cm.values.sum()

In [ ]:
acc

0.8800437961639606

In [ ]:
prec = cm.iloc[0,0]/(cm.iloc[0,0] + cm.iloc[1,0])

In [ ]:
prec

In [ ]:
recall = cm.iloc[0,0]/(cm.iloc[0,0] + cm.iloc[0,1])

In [ ]:
recall